In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import pickle

import pandas as pd
import pyarrow as pa

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error


pd.options.mode.copy_on_write = True

In [2]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename, dtype_backend="pyarrow")

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename, dtype_backend="pyarrow")
        
    print(len(df.columns))

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    print(df.duration.std())
    
    mask = (df.duration >= 1) & (df.duration <= 60)
    print(mask.sum() /len(mask))
    
    df = df[mask]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype("str[pyarrow]")
    
    return df

In [3]:
categorical = ['PULocationID', 'DOLocationID']
target = "duration"

In [4]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

19
46.44530513776802
0.9827547930522406
19
47.26394076778834
0.979444397269143


In [5]:
dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

y_train = df_train[target].values

In [6]:
X_train.shape[1]

515

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.9861908315536585

In [8]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_val = df_val[target].values

In [9]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786409148550689